# Read the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import warnings

ratings = pd.read_csv('netflix/train_ratings_all.csv', header = None)
miss_cong = pd.read_csv('netflix/train_y_rating.csv', header = None, names = ['score'])
movies = pd.read_csv('netflix/movie_titles.csv', header = None, names = ['year', 'title'])

# Peek at the data

In [ ]:
print('Head of ratings DataFrame:')
ratings.head()

In [ ]:
print('Head of miss_cong DataFrame:')
miss_cong.head()

In [ ]:
print(f'No. of raters (rows): {ratings.shape[0]}, no. of movies (cols): {ratings.shape[1]}')
print(f'No. of miss_cong raters (rows): {miss_cong.shape[0]}, no. of movies (cols): {miss_cong.shape[1]}')

In [ ]:
print('Head of movies DataFrame:')
movies.head()

In [ ]:
print(f'No. of movies (rows): {movies.shape[0]}')

# Missing Data Exploration

## General

We'd want to put `None` where there are zeros:

In [ ]:
ratings[ratings == 0] = None

How many missing values?

In [ ]:
total_missing = np.sum(np.isnan(ratings.values))
total_obs = np.prod(ratings.shape)

print(f'Total missing values: {total_missing} out of total observations: {total_obs}')
print(f'So that\'s {total_missing / total_obs :.2f} missing')

In [ ]:
# can also work directly in pandas (in pandas 2.0 df.sum(axis=None) should work)
pd.isna(ratings).sum().sum()

## Focusing on Movies

See no. of missing observations per movie, add it as another column of movies:

In [ ]:
missing_per_movie = pd.isna(ratings).sum(axis=0)
movies['n_missing'] = missing_per_movie

movies.head(20)

In [ ]:
# finding n missing of a specific movie
movies.loc[movies['title'] == 'American Beauty', 'n_missing']

Finding min, max, median and mean movie per `n_missing`:

(ignoring first 14 movies with no missing at all)

In [ ]:
movies_with_missing = movies.iloc[14:, :]

def get_val_of_col_by_col(df, by, col, value):
  idx = (np.abs(df[by] - value)).idxmin() # taking id *closest* to value because mean is involved
  return df.at[idx, col]


In [ ]:
min_n_missing = np.min(movies_with_missing['n_missing'])
min_n_missing_movie = get_val_of_col_by_col(movies_with_missing, 'n_missing', 'title', min_n_missing)

max_n_missing = np.max(movies_with_missing['n_missing'])
max_n_missing_movie = get_val_of_col_by_col(movies_with_missing, 'n_missing', 'title', max_n_missing)

median_n_missing = np.median(movies_with_missing['n_missing'])
median_n_missing_movie = get_val_of_col_by_col(movies_with_missing, 'n_missing', 'title', median_n_missing)

mean_n_missing = np.mean(movies_with_missing['n_missing'])
mean_n_missing_movie = get_val_of_col_by_col(movies_with_missing, 'n_missing', 'title', mean_n_missing)

In [ ]:
print(f'movie with minimum n missing: {min_n_missing_movie}, with {min_n_missing} missing')
print(f'movie with maximum n missing: {max_n_missing_movie}, with {max_n_missing} missing')
print(f'movie with median n missing: {median_n_missing_movie}, with {median_n_missing} missing')
print(f'movie closest to mean n missing: {mean_n_missing_movie}, mean being {mean_n_missing :.2f} missing')

Let's see a histogram of missingness:

In [ ]:
sns.displot(movies['n_missing'], kde=False, rug=True)
plt.ylabel('n movies')
plt.show()

What would you say is the skewness of such a distribution?

In [ ]:
from scipy import stats

print(f'Skewness of missingness in movies:{stats.skew(movies["n_missing"]):.2f}')

## Focus on Raters

See no. of missing observations per rater, creating a new DataFrame:

In [ ]:
raters = pd.DataFrame({'rater': range(ratings.shape[0])})
missing_per_rater = pd.isna(ratings).sum(axis=1)
raters['n_missing'] = missing_per_rater

raters.head(10)

Finding min, max, median and mean rater per `n_missing`:

In [ ]:
min_n_missing = np.min(raters['n_missing'])
min_n_missing_rater = get_val_of_col_by_col(raters, 'n_missing', 'rater', min_n_missing)

max_n_missing = np.max(raters['n_missing'])
max_n_missing_rater = get_val_of_col_by_col(raters, 'n_missing', 'rater', max_n_missing)

median_n_missing = np.median(raters['n_missing'])
median_n_missing_rater = get_val_of_col_by_col(raters, 'n_missing', 'rater', median_n_missing)

mean_n_missing = np.mean(raters['n_missing'])
mean_n_missing_rater = get_val_of_col_by_col(raters, 'n_missing', 'rater', mean_n_missing)

print(f'rater with minimum n missing: {min_n_missing_rater}, with {min_n_missing} missing')
print(f'rater with maximum n missing: {max_n_missing_rater}, with {max_n_missing} missing')
print(f'rater with median n missing: {median_n_missing_rater}, with {median_n_missing} missing')
print(f'rater closest to mean n missing: {mean_n_missing_rater}, mean being {mean_n_missing:.2f} missing')

In [ ]:
# histogram
sns.displot(raters['n_missing'], kde=False, rug=True)
plt.ylabel('n raters')
plt.show()

In [ ]:
sns.displot(raters['n_missing'], kde=False, rug=True, bins=20)
plt.ylabel('n raters')
plt.show()

# Scores Exploration (ignoring missing values)

## General

Min, max, median, mean scores:

In [ ]:
print(f'min movies score: {np.nanmin(ratings.values):.2f}')
print(f'max movies score {np.nanmax(ratings.values):.2f}')
print(f'median movies score: {np.nanmedian(ratings.values):.2f}')
print(f'mean movies score: {np.nanmean(ratings.values):.2f}')

Barchart of all scores ignoring nans:

In [ ]:
sns.displot(ratings.values[~np.isnan(ratings.values)], kde=False, rug=True)
plt.ylabel('n scores')
plt.show()

In [ ]:
# same, less fancy but shorter
plt.hist(ratings.values[~np.isnan(ratings.values)], bins = 5, range=(0.5,5.5))
plt.ylabel('n scores')
plt.show()

## Mean Scores, Marginal on Movies

Let's see mean score per movie, adding it as another column of movies:

In [ ]:
movies['mean_score'] = ratings.mean(axis=0)

movies.head(14)

In [ ]:
# finding min, max, median and mean movie per mean_score
min_mean_score = np.min(movies['mean_score'])
min_mean_score_movie = get_val_of_col_by_col(movies, 'mean_score', 'title', min_mean_score)

max_mean_score = np.max(movies['mean_score'])
max_mean_score_movie = get_val_of_col_by_col(movies, 'mean_score', 'title', max_mean_score)

median_mean_score = np.median(movies['mean_score'])
median_mean_score_movie = get_val_of_col_by_col(movies, 'mean_score', 'title', median_mean_score)

mean_mean_score = np.mean(movies['mean_score'])
mean_mean_score_movie = get_val_of_col_by_col(movies, 'mean_score', 'title', mean_mean_score)

print(f'movie with minimum mean score: {min_mean_score_movie}, with mean {min_mean_score:.2f}')
print(f'movie with maximum mean score: {max_mean_score_movie}, with mean {max_mean_score:.2f}')
print(f'movie with median mean score: {median_mean_score_movie}, with mean {median_mean_score:.3f}')
print(f'movie closest to mean mean score: {mean_mean_score_movie}, mean being {mean_mean_score:.3f}')

In [ ]:
# histogram
sns.displot(movies['mean_score'], kde=False, rug=True)
plt.ylabel('n movies')
plt.show()

What would you say is the skewness of such a distribution?

In [ ]:
print(f'Skewness of mean score in movies:{stats.skew(movies["mean_score"]):.2f}')

## Mean Scores, Marginal on Raters

See mean score per rater, adding it as another column of raters:

In [ ]:
mean_score_per_rater = np.nanmean(ratings.values, axis=1)
raters['mean_score'] = mean_score_per_rater

raters.head(10)

In [ ]:
# finding min, max, median and mean rater per mean_score
min_mean_score = np.min(raters['mean_score'])
min_mean_score_rater = get_val_of_col_by_col(raters, 'mean_score', 'rater', min_mean_score)

max_mean_score = np.max(raters['mean_score'])
max_mean_score_rater = get_val_of_col_by_col(raters, 'mean_score', 'rater', max_mean_score)

median_mean_score = np.median(raters['mean_score'])
median_mean_score_rater = get_val_of_col_by_col(raters, 'mean_score', 'rater', median_mean_score)

mean_mean_score = np.mean(raters['mean_score'])
mean_mean_score_rater = get_val_of_col_by_col(raters, 'mean_score', 'rater', mean_mean_score)

print(f'rater with minimum mean score: {min_mean_score_rater}, with mean {min_mean_score:.2f}')
print(f'rater with maximum mean score: {max_mean_score_rater}, with mean {max_mean_score:.2f}')
print(f'rater with median mean score: {median_mean_score_rater}, with mean {median_mean_score:.3f}')
print(f'rater closest to mean mean score: {mean_mean_score_rater}, mean being {mean_mean_score:.3f}')

In [ ]:
# a rater with mean 5.0?
raters.loc[[957]]

In [ ]:
ratings.loc[[957]]

In [ ]:
# histogram
sns.displot(raters['mean_score'], kde=False, rug=True)
plt.ylabel('n raters')
plt.show()

What would you say is the skewness of such a distribution?

In [ ]:
print(f'Skewness of mean score in raters:{stats.skew(raters["mean_score"]):.2f}')

## Mean Scores, Marginal on Release Year

**Warning**: a lot of pandas magic ahead.

In [ ]:
# first, "attach" year to ratings:
ratings_by_year = ratings.T # transpose
ratings_by_year['year'] = movies['year'] # add new column
ratings_by_year.head(10)

In [ ]:
# Then, "melt" DataFrame, so each year will have each score in a separate line
ratings_by_year = pd.melt(ratings_by_year, id_vars=['year'], value_vars=list(range(10000)))
ratings_by_year.columns = ['year', 'rater_id', 'score']
ratings_by_year.head()

In [ ]:
# notice this a "long" DataFrame
ratings_by_year.shape

In [ ]:
# boxplots are problematic on a 1 to 5 scale...
sns.boxplot(x='year', y='score', data = ratings_by_year)
plt.show()

In [ ]:
# so we can get mean score per year and use a simple line chart
ratings_by_year = ratings_by_year.groupby('year')['score'].mean().reset_index()
ratings_by_year.columns = ['year', 'mean_score']
sns.lineplot(x='year', y='mean_score', data = ratings_by_year)
plt.show()

# Pairwise Correlations

## General

Is a movie "missingness" correlated with its mean score?

In [ ]:
sns.scatterplot(x='n_missing', y='mean_score', data = movies)
plt.show()

In [ ]:
r = pearsonr(movies['n_missing'], movies['mean_score'])[0]
print(f'Pearson r correlation: {r:.2f}')

Is a rater's "missingness" correlated with her mean score?

In [ ]:
sns.scatterplot(x='n_missing', y='mean_score', data = raters)
plt.show()

In [ ]:
r = pearsonr(raters['n_missing'], raters['mean_score'])[0]
print(f'Pearson r correlation: {r:.2f}')

In [ ]:
# it looked to me like with very high "missingness" the mean score is lower, but:
raters['n_missing_high'] = raters['n_missing'] > 50
sns.boxplot(x='n_missing_high', y='mean_score', data = raters)
plt.show()

## Marginal on Movies

Let's look at a scatterplot of two specific movies, ignoring missing observations:

In [ ]:
def scatter_movies(mov1, mov2):
  mov1_id = movies.index[movies['title'] == mov1][0]
  mov2_id = movies.index[movies['title'] == mov2][0]
  sns.scatterplot(x=mov1_id, y=mov2_id, data = ratings)
  plt.xlabel(mov1)
  plt.ylabel(mov2)
  plt.show()

In [ ]:
scatter_movies('American Beauty', 'The Patriot')

In [ ]:
# oops, either use boxplots or make dots diameter proportional to group's size
def scatter_movies(mov1, mov2):
  mov1_id = movies.index[movies['title'] == mov1][0]
  mov2_id = movies.index[movies['title'] == mov2][0]
  mov1_scores = ratings.values[:, mov1_id]
  mov2_scores = ratings.values[:, mov2_id]
  nas = np.logical_or(np.isnan(mov1_scores), np.isnan(mov2_scores))
  agg_data = pd.DataFrame({'mov1': mov1_scores[~nas], 'mov2': mov2_scores[~nas]}).groupby(['mov1', 'mov2']).size().reset_index()
  agg_data.columns = [mov1, mov2, 'count']
  sns.scatterplot(x=mov1, y=mov2, size = 'count', hue = 'count', data = agg_data, legend = False)
  plt.xlabel(mov1)
  plt.ylabel(mov2)
  plt.show()

scatter_movies('American Beauty', 'The Patriot')

In [ ]:
# look at correlation of scores of two specific movies
def corr_movies(mov1, mov2):
    mov1_id = movies.index[movies['title'] == mov1][0]
    mov2_id = movies.index[movies['title'] == mov2][0]
    mov1_scores = ratings.values[:, mov1_id]
    mov2_scores = ratings.values[:, mov2_id]
    nas = np.logical_or(np.isnan(mov1_scores), np.isnan(mov2_scores))
    return np.round(pearsonr(mov1_scores[~nas], mov2_scores[~nas])[0], 3)

corr_movies('American Beauty', 'The Patriot')

In [ ]:
scatter_movies('Pretty Woman', 'Sweet Home Alabama')

In [ ]:
corr_movies('Pretty Woman', 'Sweet Home Alabama')

Get all movies 99 * (99 - 1) / 2 pairwise correlations:

In [ ]:
movies_titles = movies['title'].values
pair_counter = 0
pair_corrs = dict()

# a nested loop is a bit cumbersome, see pd.DataFrame.corr()
for i in range(len(movies_titles) - 1):
    for j in range(i + 1, len(movies_titles)):
        mov1 = movies_titles[i]
        mov2 = movies_titles[j]
        pair_corrs[pair_counter] = {'mov1': mov1, 'mov2': mov2, 'corr': corr_movies(mov1, mov2)}
        pair_counter += 1

pair_corrs_df = pd.DataFrame.from_dict(pair_corrs, 'index')
pair_corrs_df.head(10)

In [ ]:
# pair of movies with highest correlation:
max_corr_pair_id = pair_corrs_df['corr'].idxmax()
max_pair = pair_corrs_df.iloc[max_corr_pair_id, :].values.tolist()
print(f'movie1: {max_pair[0]}, movie2: {max_pair[1]}, correlation: {max_pair[2]:.2f}')

In [ ]:
# see how the scatterplot of these movies look like
scatter_movies(max_pair[0], max_pair[1])

In [ ]:
# pair of movies with lowest correlation:
min_corr_pair_id = pair_corrs_df['corr'].idxmin()
min_pair = pair_corrs_df.iloc[min_corr_pair_id, :].values.tolist()
print(f'movie1: {min_pair[0]}, movie2: {min_pair[1]}, correlation: {min_pair[2]:.2f}')

In [ ]:
# see how the scatterplot of these movies look like
scatter_movies(min_pair[0], min_pair[1])

But beware: what would be the correlation between two movies all 10K raters chose to give a perfect rating?

In [ ]:
# histogram
sns.displot(pair_corrs_df['corr'], kde=False, rug=True)
plt.ylabel('n pairs of movies')
plt.show()

# Focus on Raters

Look at a scatterplot of two specific raters, ignoring missing observations, making marker diameter proportional to group size:

In [ ]:
def scatter_raters(rater1, rater2):
    rater1_scores = ratings.values[rater1, :]
    rater2_scores = ratings.values[rater2, :]
    nas = np.logical_or(np.isnan(rater1_scores), np.isnan(rater2_scores))
    agg_data = pd.DataFrame({'rater1': rater1_scores[~nas], 'rater2': rater2_scores[~nas]}).groupby(['rater1', 'rater2']).size().reset_index()
    agg_data.columns = [rater1, rater2, 'count']
    sns.scatterplot(x=rater1, y=rater2, size = 'count', hue = 'count', data = agg_data, legend = False)
    plt.show()

scatter_raters(0, 1)

In [ ]:
# look at correlation of two specific raters
def corr_raters(rater1, rater2):
    rater1_scores = ratings.values[rater1, :]
    rater2_scores = ratings.values[rater2, :]
    nas = np.logical_or(np.isnan(rater1_scores), np.isnan(rater2_scores))
    return np.round(pearsonr(rater1_scores[~nas], rater2_scores[~nas])[0], 3)

corr_raters(0, 1)

In [ ]:
# for looking at all 10K * (10K - 1) / 2 pairwise correlations definitely use:
# ratings.T.corr()

# we'll stick to a nested loop and sample 10K *pairs* of raters
n = 10000
pair_corrs = dict()
for i in range(n):
    rater1, rater2 = np.random.choice(range(raters.shape[0]), 2, replace = False)
    pair_corrs[i] = {'rater1': rater1, 'rater2': rater2, 'corr': corr_raters(rater1, rater2)}
pair_corrs_df = pd.DataFrame.from_dict(pair_corrs, 'index')
pair_corrs_df.head(10)

In [ ]:
# here more likely to get missing values (why?)
sum(np.isnan(pair_corrs_df['corr']))

In [ ]:
# pair of raters with highest correlation:
max_corr_pair_id = pair_corrs_df['corr'].idxmax()
max_pair = pair_corrs_df.iloc[max_corr_pair_id, :].values.tolist()
print(f'rater1: {max_pair[0]}, rater2: {max_pair[1]}, correlation: {max_pair[2]:.2f}')

In [ ]:
# see how the scatterplot of these raters look like
scatter_raters(int(max_pair[0]), int(max_pair[1]))

In [ ]:
# pair of raters with lowest correlation:
min_corr_pair_id = pair_corrs_df['corr'].idxmin()
min_pair = pair_corrs_df.iloc[min_corr_pair_id, :].values.tolist()
print(f'rater1: {min_pair[0]}, rater2: {min_pair[1]}, correlation: {min_pair[2]:.2f}')

In [ ]:
# see how the scatterplot of these raters look like
scatter_raters(int(min_pair[0]), int(min_pair[1]))

In [ ]:
# histogram
pair_corrs_no_nans = pair_corrs_df['corr'][~np.isnan(pair_corrs_df['corr'])]
sns.displot(pair_corrs_no_nans, kde=False, rug=True)
plt.ylabel('n pairs of raters')
plt.show()

# Correlation with Miss Congeniality

In [ ]:
miss_cong_scores = miss_cong['score'].values

# see scatterplot of miss congeniality against another specific movie
def scatter_miss_cong(mov1):
    mov1_id = movies.index[movies['title'] == mov1][0]
    mov1_scores = ratings.values[:, mov1_id]
    nas = np.logical_or(np.isnan(mov1_scores), np.isnan(miss_cong_scores))
    agg_data = pd.DataFrame({'mov1': mov1_scores[~nas], 'mov2': miss_cong_scores[~nas]}).groupby(['mov1', 'mov2']).size().reset_index()
    agg_data.columns = [mov1, 'Miss Congeniality', 'count']
    sns.scatterplot(x=mov1, y='Miss Congeniality', size = 'count', hue = 'count', data = agg_data, legend = False)
    plt.show()

scatter_miss_cong('Con Air')

In [ ]:
# look at correlation of miss congeniality against another specific movie
def corr_miss_cong(mov1):
    mov1_id = movies.index[movies['title'] == mov1][0]
    mov1_scores = ratings.values[:, mov1_id]
    nas = np.logical_or(np.isnan(mov1_scores), np.isnan(miss_cong_scores))
    return np.round(pearsonr(mov1_scores[~nas], miss_cong_scores[~nas])[0], 3)

corr_miss_cong('Con Air')

In [ ]:
# all Miss Congeniality's correlations with other movies
miss_cong_corrs = dict()
for i, mov1 in enumerate(movies_titles):
    miss_cong_corrs[i] = {'mov1': mov1, 'corr': corr_miss_cong(mov1)}
miss_cong_corrs_df = pd.DataFrame.from_dict(miss_cong_corrs, 'index')
miss_cong_corrs_df.head(14)

In [ ]:
# movie with highest correlation to Miss Congeniality:
max_corr_id = miss_cong_corrs_df['corr'].idxmax()
max_movie = miss_cong_corrs_df.iloc[max_corr_id, :].values.tolist()
print(f'movie with max corr to Miss Congeniality: {max_movie[0]}, correlation: {max_movie[1]:.2f}')

In [ ]:
# see how the scatterplot looks like
scatter_miss_cong(max_movie[0])

In [ ]:
# movie with lowest correlation to Miss Congeniality:
min_corr_id = miss_cong_corrs_df['corr'].idxmin()
min_movie = miss_cong_corrs_df.iloc[min_corr_id, :].values.tolist()
print(f'movie with min corr to Miss Congeniality: {min_movie[0]}, correlation: {min_movie[1]:.2f}')

In [ ]:
# see how the scatterplot looks like
scatter_miss_cong(min_movie[0])

# Miss Congeniality Distribution

First, how does the marginal distirbution of Miss Congeniality look like?

In [ ]:
sns.displot(miss_cong_scores, kde=False, rug=True)
plt.ylabel('n raters')
plt.show()

print(f'Miss Congeniality mean score is: {np.mean(miss_cong_scores):.2f}')

## Conditional on Score

Now, how does it look like conditional on scoring high (4 or 5) on another movie?

In [ ]:
def miss_cong_dist_cond_high(mov1, high_thresh = 4.0):
    mov1_id = movies.index[movies['title'] == mov1][0]
    mov1_scores = ratings.values[:, mov1_id]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        miss_cong_filtered = miss_cong_scores[mov1_scores >= high_thresh]
    sns.displot(miss_cong_filtered, kde=False, rug=True)
    plt.ylabel('n raters')
    plt.show()
    print(f'Miss Congeniality mean score is: {np.mean(miss_cong_filtered):.2f}')

In [ ]:
miss_cong_dist_cond_high('The Wedding Planner')

In [ ]:
miss_cong_dist_cond_high('Pulp Fiction')

How does it look like conditional on scoring low (1 or 2) on another movie?

In [ ]:
def miss_cong_dist_cond_low(mov1, low_thresh = 2.0):
    mov1_id = movies.index[movies['title'] == mov1][0]
    mov1_scores = ratings.values[:, mov1_id]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        miss_cong_filtered = miss_cong_scores[mov1_scores <= low_thresh]
    sns.displot(miss_cong_filtered, kde=False, rug=True)
    plt.ylabel('n raters')
    plt.show()
    print(f'Miss Congeniality mean score is: {np.mean(miss_cong_filtered):.2f}')

miss_cong_dist_cond_low('The Wedding Planner')

In [ ]:
miss_cong_dist_cond_low('Pulp Fiction')